In [ ]:
# This code is how you download the NOAA Satellite Images. (You will need these files so that you can apply RGB to them.)
# Make sure you have the AMS 2023 Environment installed
# Three ### will represent something that needs changed in the code to get a different image

In [ ]:
# Import these libraries so that the code runs smoothly

import s3fs

import requests

import datetime

import numpy as np

from pathlib import Path

import os

In [ ]:
os.chdir('')    ### You need to make your file directory have a solid path EX os.chdir('/Users/cires/Documents/Research')

In [ ]:
# Set up the directory path for the satellite images

directory_path = Path.cwd()
print(directory_path)

In [ ]:
# Activate this package

fs = s3fs.S3FileSystem(anon=True)

In [ ]:
# The GOES Satellite that views all of America is GOES 16. (We will zoom in on these files later)

bucket = 'noaa-goes16'

products_path = bucket

products = fs.ls(products_path)

for product in products:
    print(product.split('/')[-1])

In [ ]:
# Change the below times to the Day, Month, and Year you would like to view

year = 2021 ###
month = 12  ###
day = 15    ###

julian_day = datetime.datetime(year, month, day).strftime('%j')
print(julian_day)

In [ ]:
# This code will download the images you need. You will need to convert hour to your time region
# The product I have chosen views all of the United States. (You will be able to zoom into your area of intrest later)

bucket = 'noaa-goes16'
product = 'ABI-L2-MCMIPC'
year = 2021                                       ### Change this to the proper year
julian = julian_day  
hour = 20  # This is 1 pm Mountain Time           ### Change this to the proper time

data_path = bucket + '/' + product + '/'  + str(year) + '/' + str(julian).zfill(3) + '/' + str(hour).zfill(2)

files = fs.ls(data_path)

print('Total number of files:', len(files), '\n')

print('Print out the names of the first 10 and last 10 files')
for file in files[:10]:
    print(file.split('/')[-1])
for file in files[-10:]:
    print(file.split('/')[-1])

In [ ]:
# Select how long you want to view the images for
# Simplified Example: observation_times 20:01 to 20:57 every 5 minutes (See below)

observation_times = np.arange(2001,2057,5).astype(str) ###
product_name = 'MCMIPC'
for observation_time in observation_times:
    print(observation_time)
    matches = [file for file in files if (file.split('/')[-1].split('_')[3][8:12] == observation_time and file.split('/')[-1].split('-')[2] == product_name)]

    for match in matches:
        print(match.split('/')[-1])
        print('Approximate file size (MB):', round((fs.size(match)/1.0E6), 2))
    ## fs.get is the command that downloads the file

    for match in matches:
        print(match)
        print(str(directory_path / match.split('/')[-1]))
        fs.get(match, str(directory_path / match.split('/')[-1]))

In [ ]:
# You should have all of the files you need
# Now we can start working on adding RGB and enhancing your area of intrest

In [ ]:
# Import these libraries:

import xarray as xr

import numpy as np

from matplotlib import pyplot as plt

from cartopy import crs as ccrs
import cartopy.feature as cfeature

import datetime

from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Make sure you have the same path as above

directory_path = Path.cwd()

In [ ]:
# This part is kind if annoying. Enter the file you want to enhance (At the moment you have to do this step 1 at a time)
# If someone knows how to do and eneter all of the files at once please add on
# If I have time I will try to make it more automatic

file_name = 'OR_ABI-L2-MCMIPC-M6_G16_s20213492056172_e20213492058545_c20213492059037.nc' ###
file_id = directory_path / file_name

ds = xr.open_dataset(file_id, engine='netcdf4')
ds

In [ ]:
# For the next two steps you need to read in metdata 1 and 13

ds.CMI_C01.attrs

In [ ]:
ds.CMI_C13.attrs

In [ ]:
# Make sure the max and min lie between 0 and 1

print(ds['CMI_C01'].values.min(), ds['CMI_C01'].values.max())
print(ds['CMI_C02'].values.min(), ds['CMI_C02'].values.max())
print(ds['CMI_C03'].values.min(), ds['CMI_C03'].values.max())

In [ ]:
# Check the shape of your file

ds['CMI_C01'].shape, ds['CMI_C02'].shape, ds['CMI_C03'].shape

In [ ]:
# These next steps add visible color to your image
ch1 = ds.CMI_C01
ch2 = ds.CMI_C02
ch3 = ds.CMI_C03

In [ ]:
green = 0.45*ch2 + 0.1*ch3 + 0.45*ch1

In [ ]:
# Make sure this green value is in range

green.values.min(), green.values.max()

In [ ]:
tc_RGB = np.dstack([ch2, green, ch1])
print(tc_RGB.shape)

In [ ]:
# gamma makes the image brighter so that you can see it

gamma = 3
tc_RGB_gamma = np.power(tc_RGB, 1/gamma)

plt.figure()
plt.imshow(tc_RGB_gamma)
plt.show()

In [ ]:
# Now you can add RGB to your image

In [ ]:
# Define channels
ch8 = ds.CMI_C08
ch10 = ds.CMI_C10
ch12 = ds.CMI_C12
ch13 = ds.CMI_C13

In [ ]:
# Red
img = ds.CMI_C08 - ds.CMI_C10

lower_val = -26.2
upper_val = 0.6

img_clip = np.clip(img, lower_val, upper_val)
normalized_red = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Green
img = ds.CMI_C12 - ds.CMI_C13

lower_val = -43.2
upper_val = 6.7

img_clip = np.clip(img, lower_val, upper_val)
normalized_green = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Blue
img = ds.CMI_C08

lower_val = 208.5
upper_val = 244.0

img_clip = np.clip(img, lower_val, upper_val)
normalized_blue = (img_clip-lower_val)/(upper_val-lower_val)
normalized_blue_inverted = 1-normalized_blue

In [ ]:
airmass_RGB = np.dstack([normalized_red, normalized_green, normalized_blue_inverted])

In [ ]:
plt.figure()
plt.imshow(airmass_RGB)
plt.show()

In [ ]:
# It is pretty hard to see America now. Let's fix that with some more code

In [ ]:
# This will help us find our latitude

proj_var = ds.goes_imager_projection

sat_height = proj_var.perspective_point_height
semi_major = proj_var.semi_major_axis
semi_minor = proj_var.semi_minor_axis

globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)

In [ ]:
# Now run this to find your latitude

central_lon = proj_var.longitude_of_projection_origin
print(central_lon)
crs = ccrs.Geostationary(central_longitude=central_lon,satellite_height=sat_height, globe=globe)

In [ ]:
# Now we can add boundries to our image

X = ds['x']*sat_height
Y = ds['y']*sat_height
imgExtent = (X.min(), X.max(), Y.min(), Y.max())

In [ ]:
# Make sure it worked

imgExtent

In [ ]:
# If you used the above code correctly you should get an enhanced image

proj_to = crs

fig = plt.figure()
ax = plt.subplot(projection=proj_to)

ax.coastlines('10m', linewidth=2)
ax.imshow(airmass_RGB, origin='upper', extent=imgExtent, transform=crs)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)

plt.show()

In [ ]:
# It is hard to see the particular weather event you are looking for from space
# These last steps will zoom into your area of intrest and add what time this particular weather event occured

In [ ]:
ds.attrs

In [ ]:
# This gives you time and date for your image

platform = ds.platform_ID + ' ' + ds.title[0:3]

dtinfo_s = ds.time_coverage_start[0:16].replace('T',' ')
dtinfo_e = ds.time_coverage_end[0:16].replace('T',' ')

dt_scan = datetime.datetime.strptime(dtinfo_s, '%Y-%m-%d %H:%M')
date_s = dt_scan.strftime('%d %b %Y')
print(date_s)
time_s = dt_scan.strftime('%H:%M')
print(time_s)

composite = (f'Airmass RGB Composite {date_s}')

plot_title = platform + ' ' + composite + ' ' + time_s

In [ ]:
# This is what the title will be on your enhanced image

plot_title

In [ ]:
# This last step zooms into your area of intrest and downloads your enhanced image
# It is kind of a guessing game until you have your area of intrest
# The particular coordinates below zoom into CO, WY, NE, KA, SD, and the edges of border states

proj_to = crs

fig = plt.figure(figsize=(5,5))
ax = plt.subplot(projection=proj_to)
ax.set_xlim((-2600000.5, -1481770.0))  # Change this to zoom in vertically   ###
ax.set_ylim((3584175.875, 4288198.0))  # Change this to zoom in horizontally ###

ax.coastlines('10m', linewidth=2)
ax.matshow(airmass_RGB, origin='upper', extent=imgExtent, transform=crs)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)

plt.title(plot_title, size=8)

plt.show()

saved_file_name=file_name+'.png'
fig.savefig(saved_file_name, facecolor='w', dpi=300, bbox_inches='tight')

In [ ]:
# Now that you have your image you can go back up to cell 12 and change your current file to the next file you need
# Repeat until you have all the files you need so that you can make a gif image

In [ ]:
# How to make a gif image

In [ ]:
pip install pillow

In [ ]:
# Copy the name of your image and put it below (You may have to add some more lines of code. Just copy and paste.)
from PIL import Image

frames = []

frames.append(Image.open(".png"))
frames.append(Image.open(".png"))
frames.append(Image.open(".png"))

frames[0].save("NAME_OF_DUST_STORM.gif", save_all=True, append_images=frames[1:], optimize=False, duration=200, loop=0) ### Change the name so it is relevant

In [ ]:
# Congrats your gif image should have been made!